In [41]:
import pandas as pd
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

In [42]:
train = pd.read_csv(r"E:\Lomba\ANFORCOM DDSC COMPETITION 2025 - PENYISIHAN\penyisihan-ddsc25\train.csv")
test = pd.read_csv(r"E:\Lomba\ANFORCOM DDSC COMPETITION 2025 - PENYISIHAN\penyisihan-ddsc25\test.csv")


In [43]:
le = LabelEncoder()
train["label_enc"] = le.fit_transform(train["label"])

In [44]:
model_name = "indolem/indobertweet-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_ds = Dataset.from_pandas(train[["text", "label_enc"]])
test_ds = Dataset.from_pandas(test[["text"]])

train_ds = train_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)

train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label_enc"])
test_ds.set_format(type="torch", columns=["input_ids", "attention_mask"])

Map: 100%|██████████| 5000/5000 [00:00<00:00, 8981.99 examples/s]


In [45]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

df = train.copy()
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df['label_enc'],
    random_state=42
)


# Convert ke HuggingFace Dataset
train_ds = Dataset.from_pandas(train_df[['text','label_enc']].rename(columns={'label_enc':'labels'}).reset_index(drop=True))
val_ds   = Dataset.from_pandas(val_df[['text','label_enc']].rename(columns={'label_enc':'labels'}).reset_index(drop=True))
print("Before tokenization - train columns:", train_ds.column_names)

Before tokenization - train columns: ['text', 'labels']


In [46]:
train_ds = train_ds.map(tokenize, batched=True)
val_ds   = val_ds.map(tokenize, batched=True)

print("After tokenization - train columns:", train_ds.column_names)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 9032.05 examples/s]

After tokenization - train columns: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask']


In [47]:
train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [48]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(le.classes_)
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobertweet-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=2
)

In [50]:
#!pip install evaluate
import evaluate
import numpy as np

f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return f1_metric.compute(predictions=preds, references=labels, average="macro")

In [51]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

C:\Users\MSI_PC\AppData\Local\Temp\ipykernel_25176\2214961272.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\MSI_PC\anaconda3\envs\proyek_nlp\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,F1
1,1.103600,1.200389,0.505353
2,0.929000,1.149087,0.559921
3,0.722700,1.183084,0.574724
4,0.652300,1.200316,0.570247
5,0.524500,1.213695,0.567564


c:\Users\MSI_PC\anaconda3\envs\proyek_nlp\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\MSI_PC\anaconda3\envs\proyek_nlp\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\MSI_PC\anaconda3\envs\proyek_nlp\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\MSI_PC\anaconda3\envs\proyek_nlp\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=1250, training_loss=0.863339924621582, metrics={'train_runtime': 8968.8649, 'train_samples_per_second': 2.23, 'train_steps_per_second': 0.139, 'total_flos': 1315626147840000.0, 'train_loss': 0.863339924621582, 'epoch': 5.0})

In [52]:
eval_res = trainer.evaluate()
print("Validation results:", eval_res)

c:\Users\MSI_PC\anaconda3\envs\proyek_nlp\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Validation results: {'eval_loss': 1.1830843687057495, 'eval_f1': 0.5747241103265949, 'eval_runtime': 100.3721, 'eval_samples_per_second': 9.963, 'eval_steps_per_second': 0.628, 'epoch': 5.0}


In [53]:
preds = trainer.predict(test_ds)
test_pred = np.argmax(preds.predictions, axis=-1)
test_label = le.inverse_transform(test_pred)

submission = pd.DataFrame({
    "id": test["id"],
    "label": test_label
})
submission.to_csv("submission_14.csv", index=False)
submission.head()

c:\Users\MSI_PC\anaconda3\envs\proyek_nlp\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


,id,label
0,0,reformasi
1,1,sdm
2,2,sdm
3,3,pekerjaan
4,4,reformasi
